In [26]:
import math
import numpy as np
import matplotlib.pyplot as pp
% matplotlib inline

import libstempo

In [117]:
# we'll use this throughout our tests
psr = libstempo.tempopulsar('../2222.par','../2222.tim')

In [393]:
# I didn't have time to study your parameter implementation, so this is my own

class Parameter(object):
    def __init__(self,name):
        self.name = name

    # this trick lets us pass an instantiated parameter to a signal;
    # the parameter will refuse to be renamed and will return itself
    def __call__(self,name):
        return self
        
# the following two are class *factories*: they return a customized class
# then the actual parameters (with names) are obtained by instantiating the class

def Uniform(pmin,pmax):
    # this naive class definition can be improved with type, etc.
    # but will be good enough for a demo
    class Uniform(Parameter):
        def pdf(self,p):
            return 1.0 / (pmax - pmin) if (pmin < p < pmax) else 0.0
                
        def __repr__(self):
            return '"{}":Uniform({},{})'.format(self.name,pmin,pmax)
        
    return Uniform

def Normal(mu=0,sigma=1):
    class Normal(Parameter):
        norm = 1.0 / math.sqrt(2 * math.pi * sigma**2)
        
        def pdf(self,x):
            return self.norm * math.exp(-0.5*(x - mu)**2/sigma**2)

        def __repr__(self):
            return '"{}":Normal({},{})'.format(self.name,mu,sigma)        
        
    return Normal

class ConstantParameter(object):
    def __repr__(self):
        return '"{}":Constant={}'.format(self.name,self.value)        

def Constant(val):
    class Constant(Parameter,ConstantParameter):
        value = val
    
    return Constant

In [394]:
# a uniformly distributed parameter
u = Uniform(0,10)
u

__main__.Uniform.<locals>.Uniform

In [228]:
# instantiations of this parameter
u1 = u('uniform1')
u2 = u('uniform2')

In [229]:
u1

"uniform1":Uniform(0,10)

In [230]:
u1.pdf(5), u1.pdf(-1)

(0.1, 0.0)

In [231]:
n = Normal(2,5)

In [232]:
n1 = n('normal1')
n2 = n('normal2')

In [233]:
n1.pdf(2)

0.07978845608028654

In [578]:
# base signal class. Not much in it yet
class Signal(object):
    @property
    def params(self):
        # return only nonconstant parameters
        return [par for par in self._params.values() if not isinstance(par,ConstantParameter)]

    # note that we override the constant if parameter is given in dictionary?
    def get(self,parname,params={}):
        try:
            return params[self._params[parname].name]
        except KeyError:
            return self._params[parname].value
    
    def ndiag(self,params):
        return None
    
    def Fmat(self,params):
        return None
    
    def Phivec(self,params):
        return None
    
    def Phiinv(self,params):
        return None

In [590]:
# make a collection of signals from a list of signal classes
# (this is a factor; the scheme would work also by subclassing an existing
# SignalCollection class that defines all these methods...)

def SignalCollection(metasignals):
    def __init__(self,psr):
        # instantiate all the signals with a pulsar
        self._signals = [metasignal(psr) for metasignal in self._metasignals]
    
    # we may consider defining __add__ also for instantiated signals. Why not... 
    
    @property
    def params(self):
        # this would allow duplicates
        # return set(param for signal in self._signals for param in signal.params)

        # no duplicates, but expensive, so a candidate for memoization
        ret = []
        for signal in self._signals:
            for param in signal.params:
                if param not in ret:
                    ret.append(param)
        
        return ret
                    
    # there may be a smarter way to write these...
    
    def ndiag(self,params):
        ndiags = [signal.ndiag(params) for signal in self._signals]
        return sum(ndiag for ndiag in ndiags if ndiag is not None)
    
    def Fmat(self,params):
        Fmats = [signal.Fmat(params) for signal in self._signals]
        return np.hstack(Fmat for Fmat in Fmats if Fmat is not None)
    
    def Phiinv(self,params):
        Phiinvs = [signal.Phiinv(params) for signal in self._signals]
        return np.hstack(Phiinv for Phiinv in Phiinvs if Phiinv is not None)    
    
    def Phivec(self,params):
        Phivecs = [signal.Phivec(params) for signal in self._signals]
        return np.hstack(Phivec for Phivec in Phivecs if Phivec is not None)
    
    return MetaCollection('SignalCollection',(),
                          {'__init__': __init__,
                           'params': params,
                           'ndiag': ndiag,
                           'Fmat': Fmat,
                           'Phivec': Phivec,
                           'Phiinv': Phiinv,
                           '_metasignals': metasignals})

In [580]:
# since we wish to add Signal classes (not instances) together,
# we need to make them with a custom metaclass that defines __add__

class MetaSignal(type):
    def __add__(self,other):
        if isinstance(other,MetaSignal):
            return SignalCollection([self,other])
        elif isinstance(other,MetaCollection):
            return SignalCollection([self] + other._metasignals)
        else:
            raise TypeError
    
class MetaCollection(type):
    def __add__(self,other):
        if isinstance(other,MetaSignal):
            return SignalCollection(self._metasignals + [other])
        elif isinstance(other,MetaCollection):
            return SignalCollection(self._metasignals + other._metasignals)
        else:
            raise TypeError

In [581]:
# again, a factory that returns a customized class, reusable to generate
# signal instances for many pulsars

def MeasurementNoise(efac = Uniform(0.5,1.5)):
    def __init__(self,psr):
        self._params = {'efac': efac(psr.name + '_efac')}
        self._ndiag = (psr.toaerrs*1e-6)**2

    # here we assume that we pass all parameters to all the "tensor"
    # methods of the instantiated signals. An alternative is to set
    # a persistent value for each parameter. It's less functional,
    # but (often) practicality beats purity
    def ndiag(self,params):
        return self.get('efac',params)**2 * self._ndiag

    return MetaSignal('MeasurementNoise',(Signal,),
                      {'__init__': __init__,
                       'ndiag': ndiag})

In [582]:
def EquadNoise(log10_equad = Uniform(-10,-5)):
    def __init__(self,psr):
        self._params = {'log10_equad': log10_equad(psr.name + '_log10_equad')}
        self._ndiag = np.ones_like(psr.toaerrs)

    def ndiag(self,params):
        return 10.0**(2*self.get('log10_equad',params)) * self._ndiag
    
    return MetaSignal('EquadNoise',(Signal,),
                      {'__init__': __init__,
                       'ndiag': ndiag})

In [427]:
w = MeasurementNoise()

In [428]:
w1 = w(psr)

In [429]:
w1.params

["J2222-0137_efac":Uniform(0.5,1.5)]

In [426]:
w1.ndiag({'J2222-0137_efac': 1.0})[0]

3.7423676303999994e-10

In [405]:
w1.ndiag({'J2222-0137_efac': 1.5})[0]

8.4203271683999982e-10

In [406]:
x = MeasurementNoise(efac=Normal(1,0.5))

In [407]:
x1 = x(psr)

In [408]:
x1.params

dict_values(["J2222-0137_efac":Normal(1,0.5)])

In [409]:
e = EquadNoise()

In [410]:
# combining two signals!
m = w + e

In [411]:
m1 = m(psr)

In [412]:
m1._signals

In [413]:
m1.params

["J2222-0137_efac":Uniform(0.5,1.5), "J2222-0137_log10_equad":Uniform(-18,-14)]

In [467]:
# I think the relative units are wrong, by the way.
m1.ndiag({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -7})[:20]

array([  3.74246763e-10,   4.02774761e-10,   4.40598496e-10,
         5.52282090e-10,   5.50119348e-10,   1.02202249e-09,
         5.37079501e-10,   7.27822484e-10,   9.21946667e-10,
         9.30356993e-10,   3.05599398e-11,   2.33997977e-11,
         3.46914988e-11,   1.28701132e-11,   1.21719588e-11,
         1.17173466e-11,   8.18672562e-11,   5.48070062e-11,
         6.71205024e-11,   1.77442054e-11])

In [583]:
def Function(f,**kwargs):
    class Function(object):
        def __init__(self,prefix):
            self._params = {kw: arg(prefix + '_' + kw) for kw,arg in kwargs.items()}
        
        def get(self,parname,params={}):
            try:
                return self._params[parname].value
            except AttributeError:
                return params[par.name]
        
        # params could also be a standard argument here,
        # but by defining it as ** we allow multiple positional arguments
        def __call__(self,*args,**params):
            pardict = {}
            for kw,par in self._params.items():
                if par.name in params:
                    pardict[kw] = params[par.name]
                elif hasattr(par,'value'):
                    pardict[kw] = par.value
                # allow fallback to function default arguments
                    
            return f(*args,**pardict)
        
        @property
        def params(self):
            return [par for par in self._params.values() if not isinstance(par,ConstantParameter)]
    
    return Function

In [584]:
year = 365.25 * 24 * 3600

import scipy.constants as sc

year = sc.Julian_year
f1yr = 1.0/year

# unfortunately lambdas are not acceptable here because of the way we call them
# a def is not too bad though; all sampling parameters must go in as keywords
def powerlaw(f,log10_A=-16,gamma=5):
    return (10**log10_A)**2 / 12.0 / np.pi**2 * f1yr**(gamma-3) * f**(-gamma)

PowerLaw = Function(powerlaw,log10_A=Uniform(-18,-12),gamma=Uniform(1,7))

In [483]:
PowerLaw

__main__.Function.<locals>.Function

In [484]:
pw = PowerLaw('J2222-0137')

In [485]:
pw.params

["J2222-0137_log10_A":Uniform(-18,-12), "J2222-0137_gamma":Uniform(1,7)]

In [486]:
pw._params

{'gamma': "J2222-0137_gamma":Uniform(1,7),
 'log10_A': "J2222-0137_log10_A":Uniform(-18,-12)}

In [487]:
# defaults are respected
pw(1e-7,**{"J2222-0137_log10_A": -16})

8.478344795653432e-15

In [585]:
def TimingModel():
    def __init__(self,psr):
        self._params = {}
        
        self._F = psr.designmatrix()
        self._Phiinv = np.zeros(self._F.shape[1],'d')
        
    def Fmat(self,params=None):
        return self._F
    
    def Phivec(self,params=None):
        raise RuntimeError
    
    def Phiinv(self,params=None):
        return self._Phiinv
    
    return MetaSignal('TimingModel',(Signal,),
                      {'__init__': __init__,
                       'Fmat': Fmat,
                       'Phivec': Phivec,
                       'Phiinv': Phiinv})

In [506]:
t = TimingModel()

In [507]:
t1 = t(psr)

In [508]:
t1.Phiinv()

array([ 0.])

In [653]:
# start with marginalized only... my actual matrices may be wrong, since I did them very fast

def FourierBasisGP(spectrum=None,components=20):
    def __init__(self,psr):
        self._spectrum = spectrum(psr.name)
        self._params = self._spectrum._params

        self._toas = psr.stoas

        # should use a common epoch? no subtraction at all?
        self._t = 86400.0 * (self._toas - np.min(self._toas))
        self._T = np.max(self._t) - np.min(self._t)

        self._f = np.arange(1,components+1) / self._T

        self._f2 = np.zeros(2*len(self._f),'d')
        self._f2[0::2] = self._f2[1::2] = self._f

        self._F = np.zeros((len(self._t),2*len(self._f)),'d')
        for i in range(components):
            self._F[:,2*i]   = np.cos(2*math.pi*self._f[i]*self._t)
            self._F[:,2*i+1] = np.sin(2*math.pi*self._f[i]*self._t)

    def Fmat(self,params=None):
        return self._F

    # maybe we only need diagonal Phi?
    def Phivec(self,params):
        return self._spectrum(self._f2,**params)

    def Phiinv(self,params):
        return self._T / self._spectrum(self._f2,**params)
    
    return MetaSignal('FourierBasisGP',(Signal,),
                      {'__init__': __init__,
                       'Fmat': Fmat,
                       'Phiinv': Phiinv,
                       'Phivec': Phivec})

def PowerLawRedNoise(components=20):
    return FourierBasisGP(spectrum=PowerLaw,components=components)

In [488]:
f = FourierBasisGP(spectrum=PowerLaw)

In [489]:
f1 = f(psr)

In [490]:
f1.params

["J2222-0137_log10_A":Uniform(-18,-12), "J2222-0137_gamma":Uniform(1,7)]

In [491]:
f1._f

array([ 0.0010900472,  0.0021800945,  0.0032701417,  0.004360189,
        0.0054502362,  0.0065402835,  0.0076303307,  0.008720378,
        0.0098104252,  0.010900472,  0.01199052,  0.013080567,
        0.014170614,  0.015260661,  0.016350709,  0.017440756,
        0.018530803,  0.01962085,  0.020710898,  0.021800945], dtype=float128)

In [492]:
fphi = f1.Phivec({"J2222-0137_gamma": 5, "J2222-0137_log10_A": -16})

In [493]:
n = m + f

In [494]:
n1 = n(psr)

In [495]:
n1.params

["J2222-0137_efac":Uniform(0.5,1.5),
 "J2222-0137_log10_equad":Uniform(-18,-14),
 "J2222-0137_log10_A":Uniform(-18,-12),
 "J2222-0137_gamma":Uniform(1,7)]

In [496]:
n1.ndiag({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -16,
         'J2222-0137_gamma': 5, 'J2222-0137_log10_A': -16}).shape

(244,)

In [497]:
n1.Phivec({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -16,
           'J2222-0137_gamma': 5, 'J2222-0137_log10_A': -16}).shape

(40,)

In [498]:
n1.Fmat({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -16,
         'J2222-0137_gamma': 5, 'J2222-0137_log10_A': -16}).shape

(244, 40)

In [616]:
import scipy.linalg as sl

class likelihood(object):
    def __init__(self,psr,signals):
        self._model = signals(psr)
        
        self._data = psr.residuals()
        
    @property
    def params(self):
        return self._model.params
    
    def parmap(self,xs):
        return {par.name: x for par,x in zip(self.params,xs)}

    # of course this can be massively optimized
    def logl(self,xs):
        params = self.parmap(xs)
        
        Nvec = self._model.ndiag(params)
        Ninv = np.diag(1.0/Nvec)
        
        F = self._model.Fmat(params)
        p = np.dot(F.T,self._data/Nvec)
        
        Phiinv = self._model.Phiinv(params)
        K = Phiinv + np.dot(F.T,np.dot(Ninv,F)) # do better
        cf = sl.cho_factor(K)
        expval2 = sl.cho_solve(cf,p)
        
        logdetN = np.sum(np.log(Nvec))
        logdetPhi = np.sum(np.log(Phiinv[Phiinv != 0]))
        logdetK = np.sum(2*np.log(np.diag(cf[0])))

        loglike = -0.5*(logdetN + logdetPhi + logdetK) + \
                  -0.5*np.sum(self._data**2/Nvec) + 0.5*np.dot(p,expval2)
        
        return loglike

In [523]:
testpsr = libstempo.tempopulsar('/Users/vallis/Documents/ipta/enterprise/tests/data/B1855+09_NANOGrav_11yv0.gls.par',
                                '/Users/vallis/Documents/ipta/enterprise/tests/data/B1855+09_NANOGrav_11yv0.tim')

In [669]:
s = MeasurementNoise() + TimingModel() + FourierBasisGP(spectrum=PowerLaw)

l = likelihood(testpsr,s)

In [670]:
l.params

["B1855+09_efac":Uniform(0.5,1.5),
 "B1855+09_log10_A":Uniform(-18,-12),
 "B1855+09_gamma":Uniform(1,7)]

In [671]:
l.logl([1,-16,5])

LinAlgError: 121-th leading minor not positive definite